# It trains MF-DeepONet and performs Reliability analysis of Darcy equation (2D time-independent reliability analysis)
### HF data size = 10

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt
from utils import *

from timeit import default_timer

In [ ]:
torch.manual_seed(10)
np.random.seed(10)

# **Deeponet**

In [ ]:
class Deeponet(nn.Module):
    def __init__(self, branchnetdepth, trunknetdepth, width, insize, space_dim):
        super(Deeponet, self).__init__()

        inp_dim = insize
        s_dim = space_dim
        tlayers = []
        blayers = [] 
        for i in range(branchnetdepth):
            blayers.append(nn.Linear(inp_dim,width)) 
            blayers.append(nn.ReLU(inplace=True))
            inp_dim = width
        for i in range(trunknetdepth):
            tlayers.append(nn.Linear(s_dim,width)) 
            tlayers.append(nn.ReLU(inplace=True))
            s_dim = width
        
        self.branchnet = nn.Sequential(*blayers)
        self.trunknet = nn.Sequential(*tlayers)
        self.bias = nn.Linear(1,1)
  
    def hadprodsum(self, branch, trunk):
        return torch.einsum("ij,ij->i", branch, trunk)

    def forward(self, xb,xt):
        #pdb.set_trace()
        x1 = self.branchnet(xb)
        x2 = self.trunknet(xt)
        x  = self.hadprodsum(x1,x2)
        x = x.view(-1,1)
        x  = self.bias(x)      
        return x

# Multifidelity

In [ ]:
#  configurations

ntrain_m = 10
ntest_m = 40
nreliability = 2000
last_m = 600
n_total = ntrain_m + ntest_m

epochs = 250
batch_size = 1000
learning_rate = 0.001
r = 2
s = 51


In [ ]:
# %%
""" Read data """
PATH = 'data/Darcy_Triangular_FNO_multifid_hmax018_hmin016.mat'
reader = MatReader(PATH)

x_train = np.array(reader.read_field('boundCoeff')[:,::r,::r][:,:s,:s])
y_train = np.array(reader.read_field('sol')[:,::r,::r][:,:s,:s])
y_train_l = np.array(reader.read_field('lressol')[:,::r,::r][:,:s,:s])
x_or_h = x_train[last_m-n_total:last_m].reshape((n_total,s,s,1))
y_or_h = y_train[last_m-n_total:last_m]
y_or_l = y_train_l[last_m-n_total:last_m].reshape((n_total,s,s,1))
x_coord = np.array(reader.read_field('coord_x'))[0][::r,::r][:s,:s]
y_coord = np.array(reader.read_field('coord_y'))[0][::r,::r][:s,:s]


In [ ]:
coords = np.stack((x_coord.flatten(),
                   y_coord.flatten()), axis=-1)

print(coords.shape)

In [ ]:
x_mf = np.concatenate((x_or_h, y_or_l),axis=-1)
y_mf = y_or_h - y_or_l.reshape((n_total,s,s))


In [ ]:
print(x_mf.shape, y_mf.shape)

In [ ]:
# Split the training and testing datasets

x_train_mf, y_train_mf = x_mf[:ntrain_m, ...], y_mf[:ntrain_m, ...]
x_test_mf, y_test_mf = x_mf[-ntest_m:, ...], y_mf[-ntest_m:, ...]


In [ ]:
print(x_train_mf.shape, y_train_mf.shape, x_test_mf.shape, y_test_mf.shape)


In [ ]:
# Read data:

xb_train_mf  = np.repeat(x_train_mf.astype(np.float32).reshape((ntrain_m,-1)), coords.shape[0],axis=0)
xt_train_mf =  np.tile(coords.astype(np.float32), (ntrain_m,1))
y_train_mf  =  y_train_mf.astype(np.float32).reshape(-1,1)

xb_test_mf  = np.repeat(x_test_mf.astype(np.float32).reshape((ntest_m,-1)), coords.shape[0],axis=0)
xt_test_mf =  np.tile(coords.astype(np.float32), (ntest_m,1))
y_test_mf  =  y_test_mf.astype(np.float32).reshape(-1,1)

print(xb_train_mf.shape, xt_train_mf.shape, y_train_mf.shape)
print(xb_test_mf.shape, xt_test_mf.shape, y_test_mf.shape)

xb_train_mf = torch.from_numpy(xb_train_mf)
xt_train_mf = torch.from_numpy(xt_train_mf)
xb_test_mf = torch.from_numpy(xb_test_mf)
xt_test_mf = torch.from_numpy(xt_test_mf)
y_train_mf = torch.from_numpy(y_train_mf)
y_test_mf = torch.from_numpy(y_test_mf)

train_loader_mf = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(xb_train_mf, xt_train_mf, y_train_mf),
                                              batch_size=batch_size, shuffle=True)
test_loader_mf = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(xb_test_mf, xt_test_mf, y_test_mf),
                                             batch_size=batch_size, shuffle=False)


In [ ]:
y_train_mf.shape

In [ ]:
xb_test_mf.shape

In [ ]:
# model
bdepth = 3
tdepth = 3
width = 20

inputsizeb = xb_train_mf.shape[-1]
spdim = xt_train_mf.shape[-1]

model = Deeponet(bdepth, tdepth, width, inputsizeb, spdim).cuda()
print(count_params(model))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    train_l2 = 0
    for xb,xt, y in train_loader_mf:
        xb,xt, y = xb.cuda(), xt.cuda(), y.cuda()
        
        optimizer.zero_grad()
        out = model(xb, xt)

        mse = F.mse_loss(out.view(out.shape[0], -1), y.view(out.shape[0], -1), reduction='mean')
        mse.backward() # use the l2 relative loss

        optimizer.step()
        train_mse += mse.item()

    model.eval()
    test_mse = 0
    with torch.no_grad():
        for xb,xt, y in test_loader_mf:
            xb,xt, y = xb.cuda(), xt.cuda(), y.cuda()

            out = model(xb, xt)
            tmse = F.mse_loss(out.view(out.shape[0], -1), y.view(out.shape[0], -1), reduction='mean')
            test_mse += tmse.item()

    train_mse /= len(train_loader_mf)
    test_mse /= len(test_loader_mf)
    t2 = default_timer()
    print(f'epoch {ep}, time_taken: {t2-t1}, train_mse: {train_mse},test_mse: {test_mse}')


In [ ]:
# Save the MF-DeepONet model

torch.save(model, 'model/deeponet_darcy_10')

In [ ]:
batch = 100
data_range = np.arange(0,nreliability,batch)

pred_mf = []
actual = []
x_test_mf_ = []
for i in range(len(data_range)):
    # print(data_range[i], data_range[i]+100)

    x_or_h_rel = x_train[data_range[i]:data_range[i]+batch, ...].reshape((batch,s,s,1))
    y_or_h_rel = y_train[data_range[i]:data_range[i]+batch, ...]
    y_or_l_rel = y_train_l[data_range[i]:data_range[i]+batch, ...].reshape((batch,s,s,1))

    x_mf_rel = np.concatenate((x_or_h_rel, y_or_l_rel),axis=-1)
    y_mf_rel = y_or_h_rel - y_or_l_rel.reshape((batch,s,s))
    
    # Prepare dataset for reliability:
    x_test_mf_rel, y_test_mf_rel = x_mf_rel[:batch, ...], y_mf_rel[:batch, ...]
    
    xb_test_mf_rel  = np.repeat(x_test_mf_rel.astype(np.float32).reshape((batch,-1)), coords.shape[0], axis=0)
    xt_test_mf_rel =  np.tile(coords.astype(np.float32), (batch,1))
    y_test_mf_rel  =  y_test_mf_rel.astype(np.float32).reshape(-1,1)
    
    xb_test_mf_rel = torch.from_numpy(xb_test_mf_rel)
    xt_test_mf_rel = torch.from_numpy(xt_test_mf_rel)
    y_test_mf_rel = torch.from_numpy(y_test_mf_rel)
    
    test_loader_mf_rel = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(xb_test_mf_rel, xt_test_mf_rel, y_test_mf_rel),
                                                     batch_size=5000, shuffle=False)

    with torch.no_grad():
        index = 0
        for xb, xt, y in test_loader_mf_rel:
            xb, xt, y = xb.cuda(), xt.cuda(), y.cuda()
    
            out = model(xb, xt)
            tmse = F.mse_loss(out.view(out.shape[0], -1), y.view(out.shape[0], -1), reduction='mean')
            test_mse += tmse.item()
    
            pred_mf.append( out.cpu() )
            actual.append( y.cpu() )
            input
            print("Data-Range-{}, Batch-{}, Test-loss-{:0.6f}".format( data_range[i], index, tmse ))
            index += 1

    x_test_mf_.append(x_test_mf_rel)

x_test_mf_ = np.vstack(( x_test_mf_ ))
actual = torch.cat(( actual ))
pred_mf = torch.cat(( pred_mf ))
print('Mean mse_mf-{}'.format(F.mse_loss(actual, pred_mf).item()))
    

In [ ]:
pred_mf = pred_mf.reshape(nreliability, s, s)
actual = actual.reshape(nreliability, s, s)

print(pred_mf.shape, actual.shape)

In [ ]:
out_actual = pred_mf + torch.from_numpy(x_test_mf_[:,:,:,1]) 
real_actual = actual + torch.from_numpy(x_test_mf_[:,:,:,1]) 


In [ ]:
mse_pred = F.mse_loss(out_actual, real_actual).item()

print('MSE-Predicted solution-{:0.6f}'.format(mse_pred))


In [ ]:
# Compute error statistics

error = (real_actual - out_actual)**2
mse_mean = torch.mean(error)
mse_std = torch.std(error)

print('MSE_mean-{}, MSE-std-{}'.format(mse_mean, mse_std))


In [ ]:
fig1, axs = plt.subplots(nrows=3, ncols=5, figsize=(16, 6), facecolor='w', edgecolor='k')
fig1.subplots_adjust(hspace=0.35, wspace=0.2)

fig1.suptitle(f'Predictions MFWNO AC2d Size', fontsize=16)
index = 0 
for sample in range(nreliability):
    if sample % 400 == 0:
        im = axs[0, index].imshow(real_actual[sample, :, :], cmap='nipy_spectral', origin='lower' )
        plt.colorbar(im, ax=axs[0, index])
        im = axs[1, index].imshow(out_actual[sample, :, :], cmap='nipy_spectral', origin='lower' )
        plt.colorbar(im, ax=axs[1, index])
        im = axs[2, index].imshow(torch.abs(real_actual[sample, :, :] - out_actual[sample, :, :]),
                                    cmap='jet', origin='lower')
        plt.colorbar(im, ax=axs[2, index])
        index += 1
        

# First passage failure

In [ ]:
# %%
eh = 2.2
eh_deeponet_mf = np.zeros(nreliability)

for i in range(nreliability):
    if len( np.where( out_actual[i, ...] > eh )[0] ) == 0:
        eh_deeponet_mf[i] = 0
    else:
        eh_deeponet_mf[i] = 1
        
pf_deeponet_mf = len(np.where(eh_deeponet_mf!=0)[0])/nreliability
print('Prob. of failure, MFDeepONet-{}'.format(pf_deeponet_mf))


In [ ]:
scipy.io.savemat('data/deeponet_darcy_n10.mat', mdict={'out_actual':out_actual.cpu().numpy(), 
                                                        'real_actual':real_actual.cpu().numpy(),
                                                        'x_coord':x_coord,
                                                        'y_coord':y_coord})
